# Data Reading

In [1]:
import pandas as pd

In [2]:
def load_data(prefix):
    def fix_path(src):
        return prefix + str('/IMG/') +src.split('/')[-1]    
    
    data = pd.read_csv( prefix + str('/driving_log.csv'), header=None, skiprows=1, names=[
        'center_img_path', 
        'left_img_path',
        'right_img_path',
        'steering_angle',
        'throttle',
        'break',
        'speed'
    ])
    
    data['center_img_path'] =  data['center_img_path'].apply(fix_path)
    data['left_img_path'] =  data['left_img_path'].apply(fix_path)
    data['right_img_path'] =  data['right_img_path'].apply(fix_path)

    return data

In [3]:
datasets = [load_data(prefix) for prefix in  [ './data' ]] #, './train', './train_2'] ]

In [4]:
data = pd.concat( datasets )
data.head(3)

,center_img_path,left_img_path,right_img_path,steering_angle,throttle,break,speed
0,./data/IMG/center_2016_12_01_13_30_48_287.jpg,./data/IMG/left_2016_12_01_13_30_48_287.jpg,./data/IMG/right_2016_12_01_13_30_48_287.jpg,0.0,0.0,0.0,22.148290
1,./data/IMG/center_2016_12_01_13_30_48_404.jpg,./data/IMG/left_2016_12_01_13_30_48_404.jpg,./data/IMG/right_2016_12_01_13_30_48_404.jpg,0.0,0.0,0.0,21.879630
2,./data/IMG/center_2016_12_01_13_31_12_937.jpg,./data/IMG/left_2016_12_01_13_31_12_937.jpg,./data/IMG/right_2016_12_01_13_31_12_937.jpg,0.0,0.0,0.0,1.453011


In [5]:
data.to_pickle( 'loaded_data.pkl' )

# Data Generation

In [6]:
import pandas as pd
import numpy as np
import cv2


def generator(batch_size, data):
    def load_img(src):
        return cv2.imread(src)

    correction = 0.2
    while True:
        subsample = data.sample(batch_size)
        
        subsample['center_img'] = subsample.center_img_path.apply(load_img)
        #subsample['left_img'] = subsample.left_img_path.apply(load_img)
        #subsample['right_img'] = subsample.right_img_path.apply(load_img)
    
        
        X_center = np.asarray( subsample.center_img.values.tolist() )
        y_center = subsample[['steering_angle']].values
        
        X_center_flipped = np.fliplr(X_center)
        y_center_flipped = subsample[['steering_angle']].values * (-1)

        #X_left = np.asarray( subsample.left_img.values.tolist() )
        #y_left = subsample[['steering_angle']].values + correction

        #X_right = np.asarray( subsample.right_img.values.tolist() )
        #y_right = subsample[['steering_angle']].values - correction

        X = np.concatenate( [X_center, X_center_flipped], axis=0)
        y = np.concatenate( [y_center, y_center_flipped], axis=0)
        
        yield X, y

In [7]:
data = pd.read_pickle( 'loaded_data.pkl' )

validation_data = data.sample( int(data.shape[0] * 0.1) )
train_data = data.drop(validation_data.index)

#X_valid, y_valid = next(generator(validation_data.shape[0], validation_data))


# Neural Net

In [8]:
import os

In [9]:
def preprocess_image(img):
    from keras.backend import tf as ktf
    import keras.backend as K
    
    ktf.image.resize_images(img, (80, 160))
    return (K.identity(img) / 255.0) - 0.5


In [10]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, Lambda, Cropping2D, Dropout
from keras.applications import VGG16
from keras.regularizers import l2

tb = keras.callbacks.TensorBoard(log_dir='./logs')

vgg = VGG16( include_top=False, weights='imagenet' )
vgg.trainable = False

model = Sequential()
model.add( Cropping2D(cropping=((40,20), (0,0)), input_shape=(160,320,3)) )
model.add( Lambda(preprocess_image) )
model.add( vgg )
model.add( Flatten() )
model.add( Dense(1024, activation='relu'))#, kernel_regularizer=l2(0.1) ))
model.add( Dense(128))#, activation='relu'))#, kernel_regularizer=l2(0.0001) ))
model.add( Dense(64))#, activation='relu'))#, kernel_regularizer=l2(0.0001) ))
model.add( Dense(16))#, activation='relu'))#, kernel_regularizer=l2(0.0001) ))
model.add( Dense(1) )

model.compile( optimizer='adam', loss='mse' )
model.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cropping2d_1 (Cropping2D)    (None, 100, 320, 3)       0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 100, 320, 3)       0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 15360)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              15729664  
_________________________________________________________________
dense_2 (Dense)              (None, 128)               131200    
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
__________

In [12]:
batch_size=64
steps_per_epoch = train_data.shape[0] // batch_size 

model.fit_generator( generator=generator(batch_size, train_data), 
                    steps_per_epoch=steps_per_epoch, 
                    epochs=10, 
                    callbacks=[tb],
                    validation_data=generator(batch_size, validation_data),
                    validation_steps=steps_per_epoch // 10
                   )


Epoch 1/10
113/113 [==============================] - 53s - loss: 4.0639 - val_loss: 0.0102
Epoch 2/10
113/113 [==============================] - 53s - loss: 0.0101 - val_loss: 0.0093
Epoch 3/10
113/113 [==============================] - 52s - loss: 0.0082 - val_loss: 0.0099
Epoch 4/10
113/113 [==============================] - 52s - loss: 0.0073 - val_loss: 0.0080
Epoch 5/10
113/113 [==============================] - 53s - loss: 0.0074 - val_loss: 0.0089
Epoch 6/10
113/113 [==============================] - 52s - loss: 0.0061 - val_loss: 0.0094
Epoch 7/10
113/113 [==============================] - 52s - loss: 0.0068 - val_loss: 0.0102
Epoch 8/10
113/113 [==============================] - 53s - loss: 0.0072 - val_loss: 0.0076
Epoch 9/10
113/113 [==============================] - 52s - loss: 0.0066 - val_loss: 0.0076
Epoch 10/10
113/113 [==============================] - 53s - loss: 0.0062 - val_loss: 0.0082


In [13]:
model.save( '2017-07-28.h5' )